# Day 7: Handy Haversacks

Today, we're given a problem about bags within bags. _Lots_ of bags within bags.

To begin with, we have a file of text, with each line containing a rule for how many bags of what colors are within a certain color  of bag.

The example is:
```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```

To begin with, we'll need to parse our input into something we can get more of a handle on programatically.

In [1]:
#!fsharp
type Bag = Bag of string
let (|Bag|_|) = function
  | color1 :: color2 :: ("bag" | "bags") :: rest -> Some (Bag.Bag $"{color1} {color2}", rest)
  | _ -> None

let (|Int|_|) (s:string) = 
  try s |> int |> Some with _ -> None

let rec (|BagContents|_|) = function
    | Int i :: Bag (bag, BagContents rest) -> Some ((bag, i) :: rest)
    | ["no"; "other"; "bags"] | [] -> Some []
    | _ -> None

let parseRule (line:string) =
  match line.Split([|' '; ','; '.'|], System.StringSplitOptions.RemoveEmptyEntries) |> Array.toList with
  | Bag (bag, "contain" :: BagContents contents) -> (bag, contents)
  | e -> failwith <| String.Join (' ', e)

let parseRules (inp:string) = 
  inp.Split('\n', System.StringSplitOptions.RemoveEmptyEntries)
  |> Array.map parseRule
  |> Map.ofArray

let exampleRules = @"light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags." |> parseRules
exampleRules

key,value
"Bag ""bright white""","[ ( Bag ""shiny gold"", 1 ) ]"
"Bag ""dark olive""","[ ( Bag ""faded blue"", 3 ), ( Bag ""dotted black"", 4 ) ]"
"Bag ""dark orange""","[ ( Bag ""bright white"", 3 ), ( Bag ""muted yellow"", 4 ) ]"
"Bag ""dotted black""",[ ]
"Bag ""faded blue""",[ ]
"Bag ""light red""","[ ( Bag ""bright white"", 1 ), ( Bag ""muted yellow"", 2 ) ]"
"Bag ""muted yellow""","[ ( Bag ""shiny gold"", 2 ), ( Bag ""faded blue"", 9 ) ]"
"Bag ""shiny gold""","[ ( Bag ""dark olive"", 1 ), ( Bag ""vibrant plum"", 2 ) ]"
"Bag ""vibrant plum""","[ ( Bag ""faded blue"", 5 ), ( Bag ""dotted black"", 6 ) ]"


The first question we're asked is: `"How many colors can, eventually, contain at least one shiny gold bag?`

In [1]:
#!fsharp
let allContaining target rules =
  let lookup = (rules |> Map.map (fun _ v -> v |> Seq.map fst |> Set.ofSeq)) 
  let rec containsDeep value key = 
    match Map.find key lookup with
    | contents when contents |> Set.isEmpty -> false
    | contents when contents |> Set.contains value -> true
    | contents -> contents |> Set.exists (containsDeep value)
  rules |> Map.toSeq |> Seq.map fst |> Seq.filter (containsDeep target)

exampleRules |> allContaining (Bag ("shiny gold"))

index,Item
0,bright white
1,dark orange
2,light red
3,muted yellow


That's the correct list according to the example. Now to run it on the (much larger) input file:  

In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value
let fullRules = parseRules input
fullRules |> allContaining (Bag "shiny gold") |> Seq.length

112

The second question is `How many individual bags are required inside your single shiny gold bag?`

In [1]:
#!fsharp
let rec countContents bag rules =
  match Map.find bag rules with
  | [] -> 0
  | contents -> contents |> List.sumBy (fun (b,i) -> i * (1 + countContents b rules))

"""shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.""" |> parseRules |> countContents (Bag "shiny gold")

126

In [1]:
#!fsharp
fullRules |> countContents (Bag "shiny gold")

6260